In [ ]:
import os
import boto3
import pandas as pd
from datetime import datetime

# AWS S3 credentials and bucket details
AWS_ACCESS_KEY_ID = 'your_access_key'
AWS_SECRET_ACCESS_KEY = 'your_secret_key'
AWS_REGION = 'your_region'
BUCKET_NAME = 'your_bucket_name'

# Initialize S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)

# Local folder containing files (images/docs)
local_folder = 'path_to_your_local_folder'

# Initialize an empty list to store file names and URLs
file_data = []

# Function to upload a file to S3 and return the public URL
def upload_file_to_s3(file_path, bucket_name):
    file_name = os.path.basename(file_path)
    try:
        s3.upload_file(file_path, bucket_name, file_name)
        
        # Generate the file's URL
        file_url = f"https://{bucket_name}.s3.{AWS_REGION}.amazonaws.com/{file_name}"
        return file_name, file_url
    except Exception as e:
        print(f"Error uploading {file_name}: {str(e)}")
        return None, None

# Iterate through files in the local folder and upload each to S3
for root, dirs, files in os.walk(local_folder):
    for file in files:
        file_path = os.path.join(root, file)
        file_name, file_url = upload_file_to_s3(file_path, BUCKET_NAME)
        
        if file_name and file_url:
            # Append the file name and URL to the list
            file_data.append([file_name, file_url])

# Create a DataFrame to save data into an Excel file
df = pd.DataFrame(file_data, columns=['File Name', 'File URL'])

# Save the DataFrame to an Excel file
output_excel_path = 'uploaded_files_links.xlsx'
df.to_excel(output_excel_path, index=False)

print(f"File URLs have been saved to {output_excel_path}")
